In [1]:
from proteins2info import InfoRetrieval

/Users/fabian/miniconda3/envs/auto-db-pipeline/lib/python3.8/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [2]:
genbank = InfoRetrieval()
len(genbank.entries)

11493

In [3]:
genbank.filter_AB_entries()

Number of entires removed by antibody filter: 5166
Number of entires after antibody filter: 6327
----------


In [4]:
genbank.classify_vh_vl_anarci()

In [5]:
genbank.find_antigen({})

Number of entries where antigen was determined: 0
Number of entries where antigen was not determined: 6327
----------


In [6]:
genbank.find_fragment_id()

Number of entries where fragement name was determined: 1733
Number of entries where fragment name was not determined: 4594
----------


In [7]:
genbank.find_nanobodies()

In [8]:
genbank.seperate_nanobodies()

Number of nanobodies: 155
Number of antibodies: 6172
----------


In [9]:
genbank.group_by_publication()

In [10]:
genbank.pair_vh_vl()

Number of sequence pairs: 766
Number of sequences that could not be paired in attempt 1: 4640
----------


In [11]:
genbank.get_pairable_sequences_from_sabdab()

Number of entries attempted to pair with SAbDab: 164
Number of entries not pairable with SAbDab: 4476
----------


In [12]:
genbank.pair_vh_vl_pdb()

Number of pairs found with SAbDab: 82
Number of sequences not paired but sequence from PDB added: 0


In [13]:
len(genbank.unpaired_entries)

61

In [64]:
genbank.unpaired_entries[0][0]

{'GBSeq_locus': 'UGO91527',
 'GBSeq_moltype': 'AA',
 'GBSeq_update-date': '15-DEC-2021',
 'GBSeq_create-date': '15-DEC-2021',
 'GBSeq_definition': 'immunoglobulin heavy chain variable region, partial [Homo sapiens]',
 'GBSeq_accession-version': 'UGO91527.1',
 'GBSeq_source': 'Homo sapiens (human)',
 'GBSeq_sequence': 'qvqlqesgpglvkpsetlsltctvsggsissyywswirqppgkglewigyiyysgstnynpslksrvtisvdtsknqfslklssvtaadtavyycardirdssgwanwfdpwgqgtlvtvss',
 'GBSeq_references': [{'GBReference_authors': ['Cho,H.',
    'Gonzales-Wartz,K.K.',
    'Huang,D.',
    'Yuan,M.',
    'Peterson,M.',
    'Liang,J.',
    'Beutler,N.',
    'Torres,J.L.',
    'Cong,Y.',
    'Postnikova,E.',
    'Bangaru,S.',
    'Talana,C.A.',
    'Shi,W.',
    'Yang,E.S.',
    'Zhang,Y.',
    'Leung,K.',
    'Wang,L.',
    'Peng,L.',
    'Skinner,J.',
    'Li,S.',
    'Wu,N.C.',
    'Liu,H.',
    'Dacon,C.',
    'Moyer,T.',
    'Cohen,M.',
    'Zhao,M.',
    'Lee,F.E.',
    'Weinberg,R.S.',
    'Douagi,I.',
    'Gross,R.',
    'Schm

In [96]:
import pandas as pd
df = pd.read_csv('sabdab_summary_all.tsv', sep='\t')
df[df.pdb == '7srs']

In [97]:
df.head()

,pdb,Hchain,Lchain,model,antigen_chain,antigen_type,antigen_het_name,antigen_name,short_header,date,...,scfv,engineered,heavy_subclass,light_subclass,light_ctype,affinity,delta_g,affinity_method,temperature,pmid
0,7srr,E,e,0,B | C,protein | protein,NA | NA,g protein subunit q (gi2-mini-gq chimera) | gu...,MEMBRANE PROTEIN,09/21/22,...,True,True,unknown,unknown,unknown,None,None,None,None,None
1,7srs,Q,P,0,R,protein,NaN,5-hydroxytryptamine receptor 2b,MEMBRANE PROTEIN,09/21/22,...,False,True,IGHV1,IGKV4,Kappa,None,None,None,None,None
2,7srs,H,L,0,C,protein,NaN,isoform 1b of beta-arrestin-1,MEMBRANE PROTEIN,09/21/22,...,False,True,IGHV3,IGKV1,Kappa,None,None,None,None,None
3,7ut3,H,L,0,L,Hapten,TI0,4-nitrophenyl 2-acetamido-2-deoxy-alpha-D-gala...,IMMUNE SYSTEM,09/21/22,...,False,True,IGHV5,IGKV4,Kappa,None,None,None,None,None
4,7v5j,E,D,0,A,protein,NaN,spike glycoprotein,PROTEIN BINDING,09/21/22,...,False,True,IGHV1,IGKV1,Kappa,None,None,None,None,None


In [105]:
df[df.pdb == 'abcd']

,pdb,Hchain,Lchain,model,antigen_chain,antigen_type,antigen_het_name,antigen_name,short_header,date,...,scfv,engineered,heavy_subclass,light_subclass,light_ctype,affinity,delta_g,affinity_method,temperature,pmid


In [86]:
from Bio.PDB import PDBList, PDBParser, MMCIFParser
from Bio.SeqUtils import seq1
import re

In [66]:
pdb = PDBList(pdb='./pdbs/')

In [75]:
pdb = PDBList(pdb='./pdbs/')
pdb.retrieve_pdb_file('7srs', pdir='./pdbs')

'./pdbs/7srs.cif'

In [76]:
parser = MMCIFParser()
structure = parser.get_structure('7srs', "./pdbs/7srs.cif")

/Users/fabian/miniconda3/envs/auto-db-pipeline/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain R is discontinuous at line 8323.
  warnings.warn(


In [47]:
for model in structure:
    for chain in model:
        print(chain.id)

P
Q
R
C
H
L


In [77]:
chains = {chain.id:seq1(''.join(residue.resname for residue in chain)) for chain in structure.get_chains()}

In [78]:
chains['P']

'DIVLIQSPAIMSASPGEKVTITCSASSSVSYMHWFQQKPGTSPKLWIYSTSNLASGVPARFSGSGSGTSYSLTISRMEAEDAATYYCQQRSSYPLTFGAGTKLEIKR'

In [74]:
chains

{'B': 'VSAEDKAAAERSKMIDKNLREDGEKARRTLRLLLLGADNSGKSTIVKQGIFETKFQVDKVNFHMFDVGRDERRKWIQCFNDVTAIIFVVDSSDYNRLQEALNDFKSIWNNRWLRTISVILFLNKQDLLAEKVLAGKSKIEDYFPEFARYTTPEDATPEPGEDPRVTRAKYFIRKEFVDISTASGDGRHICYPHFTCAVDTENARRIFNDCKDIILQMNLREYNLV',
 'C': 'ELDQLRQEAEQLKNQIRDARKACADATLSQITNNIDPVGRIQMRTRRTLRGHLAKIYAMHWGTDSRLLVSASQDGKLIIWDSYTTNKVHAIPLRSSWVMTCAYAPSGNYVACGGLDNICSIYNLKTREGNVRVSRELAGHTGYLSCCRFLDDNQIVTSSGDTTCALWDIETGQQTTTFTGHTGDVMSLSLAPDTRLFVSGACDASAKLWDVREGMCRQTFTGHESDINAICFFPNGNAFATGSDDATCRLFDLRADQELMTYSHDNIICGITSVSFSKSGRLLLAGYDDFNCNVWDALKADRAGVLAGHDNRVSCLGVTDDGMAVATGSWDSFLKIWN',
 'D': 'IAQARKLVEQLKMEANIDRIKVSKAAADLMAYCEAHAKEDPLLTPVPASENPF',
 'E': 'VQLVESGGGLVQPGGSRKLSCSASGFAFSSFGMHWVRQAPEKGLEWVAYISSGSGTIYYADTVKGRFTISRDDPKNTLFLQMTSLRSEDTAMYYCVRSIYYYGSSPFDFWGQGTTLTVDIVMTQATSSVPVTPGESVSISCRSSKSLLHSNGNTYLYWFLQRPGQSPQLLIYRMSNLASGVPDRFSGSGSGTAFTLTISRLEAEDVGVYYCMQHLEYPLTFGAGTKLEL',
 'R': 'WAALLILMVIIPTIGGNTLVILAVSLEKKLQYATNYFLMSLAVADLLVGLFVMPIALLTIMFEAMWPLPLVLCPAWLFLDVLFSTASIWHLCAISVDRYIAIKKPIQA

In [196]:
def get_chains_from_sabdab(entry):
    '''
    Returns the heavy-light chain pairing of a protein entry with a
    corresponding PDB entry.

    Checks the genbank locus if it corresponds to a PDB id, then
    returns the heavy and light chain sequences for the PDB id
    from SAbDab.

    param entry: genbank protein entry
    returns (H, L): heavy and light chain sequence from SAbDab
                    if PDB entry cannot be found returns None
    '''
    seq = entry['GBSeq_sequence']
    chain = entry['chain']
    id = entry['GBSeq_locus']

    pdb_id_regex = r"""
            (
                [1-9]  # first character of PDB is 1-9
                [A-Z0-9]{3}  # remaining 3 characters of PDB are alphanumeric
            )
            [^A-Z0-9]  # no alphanumeric character after, but possibly _
            """
    pattern = re.compile(pdb_id_regex, flags=re.IGNORECASE | re.VERBOSE)
    df = pd.read_csv('sabdab_summary_all.tsv', sep='\t')

    if pattern.match(id):

        pdb_id = id.split('_')[0].lower()

        p = df[df.pdb == pdb_id]

        if len(p) > 0:


            # get chain sequences from pdb
            pdb = PDBList(pdb='./pdbs/')
            pdb.retrieve_pdb_file(pdb_id, pdir='./pdbs', overwrite=True)
            parser = MMCIFParser()
            structure = parser.get_structure(pdb_id, f'./pdbs/{pdb_id}.cif')
            chains = {chain.id:seq1(''.join(residue.resname for residue in chain)) for chain in structure.get_chains()}


            if chain == 'H':
                for index, row in p.iterrows():
        
                    try: # sometimes chains names are wrong in sabdab ignore these cases
                        if chains[row.Hchain] in seq.upper(): #This line causes it to fail sometimes
                            try:
                    
                                return (chains[row.Hchain], chains[row.Lchain])
                            except KeyError:
                    
                                return None
                    except KeyError:
                        continue
            
            elif chain == 'L':
                for index, row in p.iterrows():
                    try: # sometimes chains names are wrong in sabdab ignore these cases
                        if chains[row.Lchain] in seq.upper(): 
                            try:
                                print('g')
                                return (chains[row.Hchain], chains[row.Lchain])
                            except KeyError:
                    
                                return None
                    except KeyError:
                        continue
        else:
            return None
    else:
        return None

In [197]:
get_chains_from_sabdab(genbank.unpaired_entries[16][0])

b
7k9z
d


d
1
1


In [179]:
genbank.unpaired_entries[16][0]

{'GBSeq_locus': '7K9Z_B',
 'GBSeq_moltype': 'AA',
 'GBSeq_update-date': '28-JUL-2021',
 'GBSeq_create-date': '29-DEC-2020',
 'GBSeq_definition': 'Chain B, Fab Heavy Chain',
 'GBSeq_accession-version': '7K9Z_B',
 'GBSeq_source': 'Homo sapiens (human)',
 'GBSeq_sequence': 'qvqlvqsgaevkkpgasvkvsckasggtfstygiswvrqapgqglewmgwispnsggtdlaqkfqgrvtmtrdtststvymelsslrsedtavyycasdprddiaggywgqgtlvtvssastkgpsvfplapsskstsggtaalgclvkdyfpepvtvswnsgaltsgvhtfpavlqssglyslssvvtvpssslgtqtyicnvnhkpsntkvdkkvepksc',
 'GBSeq_references': [{'GBReference_authors': ['Rujas,E.',
    'Kucharska,I.',
    'Tan,Y.Z.',
    'Benlekbir,S.',
    'Cui,H.',
    'Zhao,T.',
    'Wasney,G.',
    'Budylowski,P.',
    'Guvenc,F.',
    'Newton,J.C.',
    'Sicard,T.',
    'Semesi,A.',
    'Muthuraman,K.',
    'Nouanesengsy,A.',
    'Prieto,K.',
    'Bueler,S.A.',
    'Youssef,S.',
    'Liao-Chan,S.',
    'Glanville,J.',
    'Christie-Holmes,N.',
    'Mubareka,S.',
    'Gray-Owen,S.D.',
    'Rubinstein,J.L.',
    'Treanor,B.',
    '

In [198]:
chains

{'H': 'QVQLVQSGAEVKKPGSSVKVSCKASGYTFTSYGISWVRQAPGQGLEWMGGIIPMFGTTNYAQKFQGRVTITADKSTSTAYMELSSLRSEDTAVYYCARDRGDTIDYWGQGTLVTVSSASTKGPSVFPLAPSSGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEP',
 'L': 'SDIQMTQSPSSLSASVGDRVTITCRASQGISNNLNWYQQKPGKAPKLLIYAASSLESGVPSRFSGSGSGTDFTLTISSLQPEDFATYYCQQGNGFPLTFGPGTKVDIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNR',
 'A': 'DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNNKNYLAWYQQKPGQPPKLLIYWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYYSTPPTFGQGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRGEC',
 'B': 'VQLVQSGAEVKKPGASVKVSCKASGGTFSTYGISWVRQAPGQGLEWMGWISPNSGGTDLAQKFQGRVTMTRDTSTSTVYMELSSLRSEDTAVYYCASDPRDDIAGGYWGQGTLVTVSSASTKGPSVFPLAPSSKSTTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKSC',
 'E': 'TNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNY

In [199]:
a = 'qvqlvqsgaevkkpgasvkvsckasggtfstygiswvrqapgqglewmgwispnsggtdlaqkfqgrvtmtrdtststvymelsslrsedtavyycasdprddiaggywgqgtlvtvssastkgpsvfplapsskstsggtaalgclvkdyfpepvtvswnsgaltsgvhtfpavlqssglyslssvvtvpssslgtqtyicnvnhkpsntkvdkkvepksc'
b = 'VQLVQSGAEVKKPGASVKVSCKASGGTFSTYGISWVRQAPGQGLEWMGWISPNSGGTDLAQKFQGRVTMTRDTSTSTVYMELSSLRSEDTAVYYCASDPRDDIAGGYWGQGTLVTVSSASTKGPSVFPLAPSSKSTTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKSC'

In [204]:
b in a.upper()

False

In [184]:
pdb = PDBList(pdb='./pdbs/')
pdb.retrieve_pdb_file('7k9z', pdir='./pdbs', overwrite=True)
parser = MMCIFParser()
structure = parser.get_structure('7k9z', f'./pdbs/7k9z.cif')
chains = {chain.id:seq1(''.join(residue.resname for residue in chain)) for chain in structure.get_chains()}


In [185]:
chains

{'H': 'QVQLVQSGAEVKKPGSSVKVSCKASGYTFTSYGISWVRQAPGQGLEWMGGIIPMFGTTNYAQKFQGRVTITADKSTSTAYMELSSLRSEDTAVYYCARDRGDTIDYWGQGTLVTVSSASTKGPSVFPLAPSSGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEP',
 'L': 'SDIQMTQSPSSLSASVGDRVTITCRASQGISNNLNWYQQKPGKAPKLLIYAASSLESGVPSRFSGSGSGTDFTLTISSLQPEDFATYYCQQGNGFPLTFGPGTKVDIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNR',
 'A': 'DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNNKNYLAWYQQKPGQPPKLLIYWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYYSTPPTFGQGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRGEC',
 'B': 'VQLVQSGAEVKKPGASVKVSCKASGGTFSTYGISWVRQAPGQGLEWMGWISPNSGGTDLAQKFQGRVTMTRDTSTSTVYMELSSLRSEDTAVYYCASDPRDDIAGGYWGQGTLVTVSSASTKGPSVFPLAPSSKSTTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKSC',
 'E': 'TNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNY

In [15]:
import pandas as pd
df = pd.read_csv('../../data/supp_info/supp_seqs.csv')

In [18]:
df.columns

Index(['Unnamed: 0', 'Name', 'Binds_to', 'Origin', 'VH', 'VL'], dtype='object')

In [2]:
import json
with open('../../data/keywords2papers/pubmed_results-2022_03_08.jsonl', 'r') as f:
    data = json.load(f)

data

JSONDecodeError: Extra data: line 2 column 1 (char 1597)

In [38]:
from pandas import read_json
import pandas as pd
import numpy as np
pubmed = read_json('../../data/keywords2papers/pubmed_results-2022_03_08.jsonl', lines=True)
biorxiv = read_json('../../data/keywords2papers/biorxiv_results-2022_03_08.jsonl', lines=True)
df = pd.concat([pubmed, biorxiv])
df.replace('', np.nan, inplace=True)
df.dropna(subset=['doi'], inplace=True)
ls = ['http://doi.org/' + x for x in df.doi.values]
ls

['http://doi.org/10.1128/spectrum.02702-21',
 'http://doi.org/10.1128/spectrum.02191-21',
 'http://doi.org/10.15252/embr.202154199',
 'http://doi.org/10.1016/j.xinn.2022.100221',
 'http://doi.org/10.1093/ofid/ofac055',
 'http://doi.org/10.3892/br.2022.1512',
 'http://doi.org/10.3389/fimmu.2022.828053',
 'http://doi.org/10.3389/fimmu.2022.811802',
 'http://doi.org/10.4155/tde-2022-0011',
 'http://doi.org/10.1016/j.ebiom.2022.103904',
 'http://doi.org/10.1016/j.cmi.2022.02.030',
 'http://doi.org/10.1002/eji.202149758',
 'http://doi.org/10.1038/s41598-022-07574-w',
 'http://doi.org/10.1038/s41467-022-28768-w',
 'http://doi.org/10.3389/fmed.2022.806611',
 'http://doi.org/10.1038/s41587-022-01232-2',
 'http://doi.org/10.1038/s41467-022-28882-9',
 'http://doi.org/10.1038/s41467-022-28766-y',
 'http://doi.org/10.1038/s41392-022-00926-y',
 'http://doi.org/10.1080/21645515.2022.2035573',
 'http://doi.org/10.1016/j.micres.2022.126993',
 'http://doi.org/10.1016/j.antiviral.2022.105271',
 'http://

In [ ]:
def papers2urls(pubmed, biorxiv):
    '''Function to get list of urls from papers'''
    df = pd.concat([pd.DataFrame(pubmed), pd.DataFrame(biorxiv)])
    df.replace('', np.nan, inplace=True)
    df.dropna(subset=['doi'], inplace=True)
    return ['http://doi.org/' + x for x in df.doi.values]


In [39]:
a = [{'a': '1', 'b': '2'}, {'a': '2', 'b': '3'}, {'a': '4', 'b': '5'}]

In [40]:
df = pd.DataFrame(a)

In [41]:
df

,a,b
0,1,2
1,2,3
2,4,5
